In [1]:
from BasisExpansion import BasisExpansion
import numpy as np
from LinClass import LinClass
from scipy.special import expit

In [2]:
#'aa' coded as 1, 'ao' coded as 0
phonemeData = np.loadtxt('cleanedPhonemeBinary.data.txt')
arrayY = phonemeData[:,-1]
y = np.zeros((len(phonemeData), 2))
for ii in range(len(phonemeData)):
    if arrayY[ii] == 1:
        y[ii] = np.array([1, 0])
    elif arrayY[ii] == 0:
        y[ii] = np.array([0, 1])
    else:
        print('something wrong')
        print(ii)
        print(y[ii])
#'aa' is coded to [1 0] and 'ao' is coded to [0 1]
X = phonemeData[:,0:-1]
print(X)
print(y)

[[12.96705 13.69454 14.91182 ...  7.036    7.01278  8.52197]
 [10.95324 11.20585 16.17634 ...  8.44448  8.28905  8.04018]
 [ 9.37324 11.29505 17.15139 ...  7.73464  7.41363  0.5371 ]
 ...
 [12.01774 11.86761 16.34707 ...  8.85308  9.88492  6.35767]
 [ 8.39388  9.8477  16.24534 ...  9.37979  6.83916  8.54817]
 [ 8.14032  9.93753 16.30187 ...  7.08945  7.72929  6.42167]]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [3]:
XTrain, XTest = X[:int(len(X)/3)], X[int(len(X)/3):]
yTrain, yTest = y[:int(len(y)/3)], y[int(len(y)/3):]
PhenomeClass = LinClass(XTrain, yTrain, ['aa', 'ao'])
phenomeRegg = PhenomeClass.binaryLogisticRegg()

331.57745313799575
138.52200217198077
59.03911049296086
23.10040598941347
8.625140260233549
3.1876570917841986
1.1767597865197235
0.43416215373070743
0.16006299722424572
0.05897293422634974
0.02171825252134446
0.007996136169998712
0.002943460495706297
0.0010833712181009608
0.0003987018889009886
0.00014671653741983652
5.3985446331408504e-05
1.986316622302537e-05
7.308052336087416e-06
2.688700531968771e-06


In [4]:
XtestStandardized = PhenomeClass.standardizeTest(XTest)
correct = 0
total = 0
for ii in range(len(XtestStandardized)):
    x = XtestStandardized[ii]
    probability = 1 - expit(-np.dot(x, phenomeRegg))
    if probability > 1/2:
        result = np.array([1, 0])
    else:
        result = np.array([0, 1])
    if (result == yTest[ii]).all():
        correct+=1 
    total +=1
print(correct/total)

0.6986899563318777


In [5]:
Expander = BasisExpansion()
expandedXTrain = np.zeros((len(XTrain), len(XTrain[0])*11))
expandedXTest = np.zeros((len(XTest), len(XTest[0])*11))
index = 0
for ii in range(len(XTrain[0])):
    expandedXTrain[:,index] = XTrain[:, ii]
    expandedXTest[:,index] = XTest[:, ii]
    index +=1
    knots = Expander.getKnots(12, XTrain[:, ii])
    for jj in range(len(knots[:-2])):
        knot = knots[jj]
        expandedXTrain[:,index] = Expander.naturalCubeSplines(XTrain[:, ii], knots, jj)
        expandedXTest[:,index] = Expander.naturalCubeSplines(XTest[:,ii], knots, jj)
        index +=1

In [6]:
print(expandedXTrain[:,-1])

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.24429808e+00 2.14437008e-03 4.67803377e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.28567193e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.61269957e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.12164531e+00 2.32345665e-02 0.00000000e+00
 0.00000000e+00 3.441131

In [10]:
ExpandedPhenomeClass = LinClass(expandedXTrain, yTrain, indicatorArray = ['aa', 'ao'])
expandedPhenomeRegg = ExpandedPhenomeClass.binaryLogisticRegg(damping= -0.01)
expandedXTestStandardized = ExpandedPhenomeClass.standardizeTest(expandedXTest)

762.152067587156
266.44712400220993
96.45450170743815
35.286528758780776
12.959794000637105
4.771207791208048
1.76178811974584
0.6534605942200894
0.24414768497008452
0.09237113025061863
0.03572749505075387
0.014123961825179651
0.005987903175512355
0.0033399450328463077
0.002468740244671259
0.0020195233207880086
0.0017155262984005443
0.001491325984008464
0.0013186450346098517
0.0011815216076352655
0.0010700188133478659
0.0009775924500842385
0.0008997508458935427
0.0008333076514568315
0.0007759398150313644
0.0007259139790169704
0.0006819107473244733
0.0006429082391858805
0.0006081028302705357
0.0005768539278436426
0.0005486446896373948
0.0005230535658226704
0.0004997333400235677
0.0004783954627138153
0.0004587981820192374
0.00044073744075328724
0.00042403981624371983
0.0004085569879116114
0.00039416136060730543
0.00038074257186424806
0.0003682046816005209
0.00035646389391241317
0.0003454466969555066
0.00033508833428307473
0.0003253315406126934
0.00031612549042419355
0.0003074249184872642

3.093944785156702e-05
3.085382061580233e-05
3.076866621009413e-05
3.068398073406501e-05
3.059976031543793e-05
3.0516001142884714e-05
3.0432699435038427e-05
3.034985145754055e-05
3.0267453502383633e-05
3.018550191737206e-05
3.0103993091555946e-05
3.0022923440960934e-05
2.9942289425052706e-05
2.9862087543816238e-05
2.9782314328901365e-05
2.9702966350641195e-05
2.962404021913915e-05
2.954553258872597e-05
2.9467440127710838e-05
2.9389759555719213e-05
2.9312487621890467e-05
2.9235621107940054e-05
2.915915682961296e-05
2.908309164708722e-05
2.9007422434723383e-05
2.8932146111537982e-05
2.885725962138416e-05
2.8782759948054358e-05
2.8708644104981282e-05
2.863490912460941e-05
2.85615520872012e-05
2.8488570083466405e-05
2.8415960256528737e-05
2.8343719757417644e-05
2.8271845775685782e-05
2.820033553691993e-05
2.8129186274738782e-05
2.8058395274603435e-05
2.7987959825940995e-05
2.791787726455911e-05
2.784814494196286e-05
2.777876023801809e-05
2.7709720564222844e-05
2.7641023345859434e-05
2.75726

1.5436323215687812e-05
1.5414991284097828e-05
1.5393718253344096e-05
1.537250388600033e-05
1.5351347930841465e-05
1.5330250154087224e-05
1.5309210308463876e-05
1.5288228161390785e-05
1.526730347623889e-05
1.5246436010290414e-05
1.5225625533423361e-05
1.5204871819617015e-05
1.5184174623192073e-05
1.5163533725139475e-05
1.51429488864422e-05
1.5122419886688966e-05
1.5101946497113258e-05
1.5081528490617735e-05
1.506116564152731e-05
1.5040857730968394e-05
1.5020604532148516e-05
1.5000405826534968e-05
1.4980261396738499e-05
1.496017101561268e-05
1.4940134477758521e-05
1.492015155848866e-05
1.4900222049739339e-05
1.4880345729445276e-05
1.4860522385060183e-05
1.4840751817706242e-05
1.4821033804124064e-05
1.4801368139047866e-05
1.478175461533764e-05
1.4762193024289256e-05
1.4742683156886504e-05
1.4723224816973136e-05
1.4703817789872929e-05
1.4684461888279055e-05
1.4665156891067769e-05
1.4645902613694534e-05
1.4626698842839617e-05
1.4607545392529242e-05
1.4588442056050422e-05
1.4569388642026195e

1.0303967755706153e-05
1.029446155090666e-05
1.0284972877208656e-05
1.0275501686936503e-05
1.0266047934909346e-05
1.0256611565533071e-05
1.0247192538437605e-05
1.0237790795768888e-05
1.022840630139799e-05
1.0219038999531862e-05
1.0209688850552707e-05
1.0200355801004545e-05
1.0191039802456239e-05
1.0181740815793119e-05
1.0172458787873986e-05
1.016319367528983e-05
1.0153945436203026e-05
1.0144714017469988e-05
1.0135499372280093e-05
1.0126301459281653e-05
1.0117120232773429e-05
1.0107955650670018e-05
1.0098807659006578e-05
1.0089676214119272e-05
1.0080561278430299e-05
1.00714628060208e-05
1.0062380746724711e-05
1.0053315052460478e-05
1.0044265695353764e-05
1.0035232617980958e-05
1.0026215779079328e-05
1.0017215136211101e-05
1.0008230643469147e-05
9.99926226295711e-06
9.990309947659253e-06
9.981373651454705e-06
9.97245333646238e-06
9.963548954861754e-06
9.954660466631371e-06
9.945787831887846e-06
9.936931004900837e-06
9.92808994529893e-06
9.91926460857225e-06
9.910454956754268e-06
9.901660

7.6538674699795e-06
7.6486222719248e-06
7.643384263014962e-06
7.638153421865158e-06
7.632929738937708e-06
7.627713198263503e-06
7.622503787996409e-06
7.617301488829139e-06
7.6121062891777335e-06
7.6069181735426946e-06
7.601737130354468e-06
7.596563140316341e-06
7.591396191270955e-06
7.586236270426233e-06
7.581083361443188e-06
7.5759374517041116e-06
7.57079852239496e-06
7.565666564782458e-06
7.56054156179858e-06
7.555423497238651e-06
7.550312367114769e-06
7.545208147951766e-06
7.540110822255185e-06
7.535020387927575e-06
7.52993682658986e-06
7.524860113993216e-06
7.519790251455052e-06
7.514727216488259e-06
7.509670998965512e-06
7.50462158086429e-06
7.499578952058702e-06
7.494543097684662e-06
7.489514006573977e-06
7.484491662977338e-06
7.479476048103338e-06
7.474467154001632e-06
7.469464975206615e-06
7.464469479632363e-06
7.459480670476516e-06
7.4544985232362944e-06
7.449523034924136e-06
7.444554183105662e-06
7.439591957748977e-06
7.434636348954365e-06
7.429687334866177e-06
7.424744909579

6.094961164700042e-06
6.091635116052833e-06
6.088312695062301e-06
6.0849938967210225e-06
6.0816787177277634e-06
6.078367150277535e-06
6.075059185366284e-06
6.0717548270350635e-06
6.068454058851917e-06
6.065156878164649e-06
6.061863280394776e-06
6.058573256820127e-06
6.055286806204841e-06
6.052003922672077e-06
6.048724596012871e-06
6.045448821294908e-06
6.04217659330445e-06
6.038907906531463e-06
6.035642758205674e-06
6.032381138410197e-06
6.02912304643355e-06
6.025868472127248e-06
6.022617408043229e-06
6.019369855772929e-06
6.016125803652715e-06
6.012885244738935e-06
6.009648178147479e-06
6.006414597001159e-06
6.003184491950041e-06
5.999957865700499e-06
5.996734703785789e-06
5.993515005511197e-06
5.990298760493053e-06
5.987085971084774e-06
5.983876627353437e-06
5.980670721183506e-06
5.977468253232478e-06
5.974269211620419e-06
5.9710735916552815e-06
5.967881392757198e-06
5.964692603527695e-06
5.9615072259087205e-06
5.958325246039214e-06
5.955146665243347e-06
5.951971475012055e-06
5.94879

5.0683503987913925e-06
5.066050526618558e-06
5.063752751125902e-06
5.061457052912914e-06
5.0591634377917226e-06
5.056871900942707e-06
5.054582441743716e-06
5.052295053814213e-06
5.050009732185945e-06
5.0477264854248635e-06
5.045445297515976e-06
5.043166168971957e-06
5.040889104987317e-06
5.0386140978498605e-06
5.036341138240994e-06
5.034070231558282e-06
5.031801375280547e-06
5.029534562927024e-06
5.027269787365872e-06
5.025007057890119e-06
5.022746365352044e-06
5.0204877036189935e-06
5.018231072549311e-06
5.01597647350039e-06
5.013723893690459e-06
5.011473342310152e-06
5.009224807801573e-06
5.006978292753053e-06
5.004733791214938e-06
5.002491303970553e-06
5.000250826768709e-06
4.998012354984034e-06


NameError: name 'expandedXtestStandardized' is not defined

In [11]:
correct = 0
total = 0
for ii in range(len(expandedXTestStandardized)):
    x = expandedXTestStandardized[ii]
    probability = 1 - expit(-np.dot(x, expandedPhenomeRegg))
    if probability > 1/2:
        result = np.array([1, 0])
    else:
        result = np.array([0, 1])
    if (result == yTest[ii]).all():
        correct+=1 
    total +=1
print(correct/total)

0.7720524017467248
